## 1단계 컨트랙 개발

In [5]:
%%writefile src/Timer.sol
pragma solidity ^0.4.25;
contract Timer {
    uint256 startTime;
    function start() public {
        startTime=block.timestamp;
    }
    function timePassed() public view returns(uint256) {
        return block.timestamp-startTime;
    }
    function getNow() view public returns(uint) {
        return block.timestamp;
    }
}

Overwriting src/Timer.sol


## 2단계 컴파일 (Josn 파일 생성)

In [6]:
!solc --optimize --combined-json abi,bin src/Timer.sol > src/Timer.json

### 확인

In [7]:
!type src\Timer.json

{"contracts":{"src/Timer.sol:Timer":{"abi":"[{\"constant\":true,\"inputs\":[],\"name\":\"timePassed\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":true,\"inputs\":[],\"name\":\"getNow\",\"outputs\":[{\"name\":\"\",\"type\":\"uint256\"}],\"payable\":false,\"stateMutability\":\"view\",\"type\":\"function\"},{\"constant\":false,\"inputs\":[],\"name\":\"start\",\"outputs\":[],\"payable\":false,\"stateMutability\":\"nonpayable\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5060df8061001f6000396000f30060806040526004361060525763ffffffff7c0100000000000000000000000000000000000000000000000000000000600035041663b445425381146057578063bbe4fd5014607b578063be9a655514608d575b600080fd5b348015606257600080fd5b50606960a1565b60408051918252519081900360200190f35b348015608657600080fd5b50606960a9565b348015609857600080fd5b50609f60ad565b005b600054420390565b4290565b426000555600a165627a7a723058208109c0055e

## 3단계 컨트랙 배포

In [11]:
%%writefile src/TimerDeployAbiBinFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./Timer.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts);
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;

_abiArray=JSON.parse(_abi);   
_bin=_abiBinJson.contracts[contractName].bin;

console.log("- ABI: " + _abiArray);
console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 364124}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/TimerDeployAbiBinFromFile.js


##### 배포된 컨트랙 주소 얻기

In [12]:
!node src/TimerDeployAbiBinFromFile.js

- contract name:  [ 'src/Timer.sol:Timer' ]
- ABI: [object Object],[object Object],[object Object]
- Bytecode: 608060405234801561001057600080fd5b5060df8061001f6000396000f30060806040526004361060525763ffffffff7c0100000000000000000000000000000000000000000000000000000000600035041663b445425381146057578063bbe4fd5014607b578063be9a655514608d575b600080fd5b348015606257600080fd5b50606960a1565b60408051918252519081900360200190f35b348015608657600080fd5b50606960a9565b348015609857600080fd5b50609f60ad565b005b600054420390565b4290565b426000555600a165627a7a723058208109c0055e95a777463c79295d8fd4ff513be864c10d599e98fea6b6b9dd38a60029
Deploying the contract from 0x2DC31BE20e440a13288b338d24113b685C456922
hash: 0xb73863683bd2d736d91f3b14e205e665e9314a5e2d8d392583b466df940b1628
---> The contract deployed to: 0x1c0702E3e5aD40Ce4EAD6178453656a1Bf2c48a7


## 4단계 사용

In [13]:
%%writefile src/timerUse.js
var Web3=require('web3');
var _abiBinJson = require('./Timer.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts);

_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abi);

var timer = new web3.eth.Contract(_abiArray,"0x1c0702E3e5aD40Ce4EAD6178453656a1Bf2c48a7");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Call from: " + accounts[0]);
    timer.methods.getNow().call().then(function(value) {console.log("Now: ", value);});
    await timer.methods.start().send({from:accounts[0],gas:100000}); 
    timer.methods.timePassed().call().then(function(value) {console.log("Passed: ", value);});
}

doIt()

Overwriting src/timerUse.js


In [14]:
!node src/timerUse.js

Call from: 0x2DC31BE20e440a13288b338d24113b685C456922
Now:  1650446764
Passed:  0
